In [410]:
import requests
import pandas as pd
import json
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from scrapy.http import TextResponse
from fake_useragent import UserAgent
import re
import string
import warnings
import time
import getpass
import time

In [146]:
# 여기서부터 시작

In [286]:
login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

driver = webdriver.Chrome()
driver.get(login_url)

In [287]:
my_id = input('id를 입력하세요: ')
my_password = getpass.getpass('password를 입력하세요: ')

driver.find_element_by_css_selector(
    '._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
driver.find_element_by_css_selector(
    '._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()

id를 입력하세요: yeejun90
password를 입력하세요: ········


In [289]:
# 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드

alert_modal = driver.find_element_by_css_selector(
    'body > div.RnEpo.Yx5HN > div > div')

if alert_modal:
    driver.find_element_by_css_selector(
        'body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()

In [295]:
keyword = '초코우유'

driver.find_element_by_css_selector(
    '#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

# 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다

# time.sleep(2)

In [296]:
search_list = driver.find_elements_by_css_selector(
    '#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')
len(search_list)

55

In [297]:
# 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
# 게시물 숫자는 문자열의 가장 마지막에 위치한다
number_list = []

for element in search_list:
    splited_element = element.text.split('게시물')
    if len(splited_element) == 2:
        number_list.append(int(re.sub(",", "", splited_element[1])))
    else:
        number_list.append(0)

max_number = max(number_list)
click_index = number_list.index(max_number)
click_index

0

In [298]:
# 클릭하고 렌더링 끝날때까지 기다리기

search_list[click_index].click()
time.sleep(1.5)

In [299]:
# 인기게시물 9개는 빼고, 최근사진부터 크롤링 시작(인기게시물과 최근게시물에 중복 게시물이 있을거 같아서)

In [301]:
# 스크롤을 현재 화면에서 인식하는 마지막 지점으로 내리면,
# 현재 스크롤 끝까지 한번 내릴때마다 최근사진3개가 담긴 div가 4개씩 추가됨
# 즉, 12개 게시물이 더 생긴다
# 스크롤 끝까지 내리고, 셀렉터로 div리스트 가져와서 마지막 4개씩 계속 크롤링하기
# 스크롤 내리고 1초정도 기다리기

# 일단 맨 처음 받아온 3개짜리 게시물이 들어있는 8개의 div를 크롤링하고
# 그 이후부터 스크롤을 끝까지 한번씩 내리면서
# div가 4개씩 들어오는것들을 크롤링한다
# 이 4개짜리 끝부분 div크롤링을 반복한다


# driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

In [302]:
# 처음 -> n = 1 ~ 8, m = 1 ~ 3
# react-root > section > main > article > div:nth-child(3) > div > div:nth-child(n) > div:nth-child(m)
# 두번째부터는 스크롤을 내리면서 n = len(사진게시물 div) - 4 + 1 ~ len(사진게시물 div) + 1, m = 1 ~ 3

In [88]:
# 클릭된 게시물을 bs4로 해시태그 크롤링하는 코드

# cur_url = driver.current_url
# response = requests.get(cur_url)
# response

# dom = BeautifulSoup(response.content, 'html.parser')
# meta_list = dom.select("meta[property='instapp:hashtags']")

# selector = #react-root > section > main > article > div:nth-child(3) > div > div > .v1Nh3.kIKUG._bz0w > a 의
# href에서 도메인 뒤에 오는 변형된 코드를 가져올 수 있다. -> scrapy에서 사용 가능할듯
# transformed_ids = ~~~.xpath(selector)
# link = 'https://instagram.com/{}'.format(transformed_ids[i])

In [305]:
# upload_time = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.k_Q0X.NnvRN > a > time')
# upload_time.text

In [306]:
# data = [
#     {'time': upload_time.text, 'hash_tag': hash_tag_list}
# ]

# df = pd.DataFrame(data)
# df

In [307]:
# 크롤링해서 가져온 데이터들을 hash_tag_df에 차곡차곡 쌓을거임
# hash_tag_df.reset_index().drop(columns='index') 다 크롤링하고 마지막에 이거 해주기

# hash_tag_df = hash_tag_df.append(df)
# hash_tag_df

In [560]:
hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
hash_tag_df

,time,hash_tag


In [561]:
# 처음 렌더링된 화면의 8개 div 크롤링하는 함수


def initial_crawling():
    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    for i in range(len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [562]:
# 스크롤내리면서 4개씩 추가되는 div 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
# 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...
# 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐

def second_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1.5)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 12, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [563]:
# 세번째부터는 이 함수 반복하기

def repeat_crawling():
    for i in range(0, 1):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1.5)

        urls = driver.find_elements_by_xpath(
            '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls) - 9, len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content']
                             for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [574]:
# 맨처음 화면의 게시물 갯수

urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

24

In [577]:
# 스크롤을 한번 끝까지 내렸을때 새로 불러온 게시물을 포함한 전체 게시물 갯수

driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
time.sleep(1.5)
urls = driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

45

In [566]:
req = requests.get(urls[0])
req.url

'https://www.instagram.com/p/B9nsG5bBnF5/'

In [568]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

response

<200 https://www.instagram.com/p/B9nsG5bBnF5/>

In [581]:
response.xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span/a')

[]

In [580]:
import pprint

pprint.pprint(req.text)

('<!DOCTYPE html>\n'
 '<html lang="en" class="no-js not-logged-in client-root">\n'
 '    <head>\n'
 '        <meta charset="utf-8">\n'
 '        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n'
 '\n'
 '        <title>\n'
 'HoJun on Instagram: “제주도 미쁜카페 우연히 지나가다 예뻐서 들어간 카페 #제주도 #미쁜카페 #우연히 #찾은 #카페 '
 '#한옥 #너무좋다 #병음료 #맛있다 #쑥쑥라떼 #제주리얼말차라떼 #초코우유#제주리얼당근주스#초코우유 #소금앙버터 #모찌모찌식빵 '
 '#제주리얼딸기라떼…”\n'
 '</title>\n'
 '\n'
 '        \n'
 '        <meta name="robots" content="noimageindex, noarchive">\n'
 '        <meta name="apple-mobile-web-app-status-bar-style" '
 'content="default">\n'
 '        <meta name="mobile-web-app-capable" content="yes">\n'
 '        <meta name="theme-color" content="#ffffff">\n'
 '        <meta id="viewport" name="viewport" content="width=device-width, '
 'initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">\n'
 '        \n'
 '        \n'
 '    <link rel="manifest" href="/data/manifest.json">\n'
 '\n'
 '        <link rel="preload" '
 'href="/stat

In [32]:
driver.quit()